## Classes and Objects

In [6]:
# In order to change the representation of an object, set methods __repr__ and __str__
# If method __str__ is not provided, __repr__ will be called as a fallback

class ClassA:
    def __init__(self, attribute):
        self.attribute_1 = attribute

In [9]:
classA_instance_1 = ClassA('attribute_1_value')
classA_instance_1

In [20]:
class ClassB(ClassA):
    def __init__(self, attribute, attribute2):
        super(ClassB, self).__init__(attribute)
        self.attribute_2 = attribute2
    
    def __repr__(self):
        return f"This is first attribute: {self.attribute_1} This is second attribute: {self.attribute_2}"


In [21]:
classB_instance_1 = ClassB('attribute_1_value', 'attribute_2_value')
classB_instance_1

This is first attribute: attribute_1_value This is second attribute: attribute_2_value

In [22]:
str(classB_instance_1)

'This is first attribute: attribute_1_value This is second attribute: attribute_2_value'

In [29]:
## Allow formatting in a class
from datetime import datetime
class ClassC(ClassB):
    def __init__(self, *args, day, month, year, **kwargs):
        super(ClassC, self).__init__(*args, **kwargs)
        self.day = day
        self.month = month
        self.year = year
        self._formats = {
            'ymd' : '{d.year}-{d.month}-{d.day}',
            'mdy' : '{d.month}/{d.day}/{d.year}',
            'dmy' : '{d.day}/{d.month}/{d.year}'
        }
    def __format__(self, code):
        if self._formats.get(code) is None:
            code = 'ymd'
        fmt = self._formats[code]
        return fmt.format(d = self)


In [33]:
my_date = datetime(2021, 11, 12)
classC_instance = ClassC(1, 2, day = 1, month = 1, year = 2021)

In [37]:
format(classC_instance), 'The date is {:mdy}'.format(classC_instance), f"The date is -> {classC_instance:dmy}"

('2021-1-1', 'The date is 1/1/2021', 'The date is -> 1/1/2021')

In [38]:
# In order to implemennt the context manager into a class, the methods __enter__ and __exit__ should
# be defined.

In [54]:
# Encapsulating private data (attributes, methods) in classes
# There is no real way of doing this, however, with the "_*" notation
# as a convention you can let the other programmers assume that method/attribute
# should not be accessed

class A:
    def __init__(self):
        self._interal = 0
        self.public = 1

    def public_method(self):
        pass

    def _private_method(self):
        pass

In [1]:
# Manage access to an attribute of an object with the Property method:
# For example you with to check wether firstname passed attribute is an string

# By doing this you cannot check the condition in the initializacion, example
class Person:
    def __init__(self, first_name):
        self._first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


guido = Person(2)

In [4]:
# In order to call the setter on the class constructor, you should call it
# when assinging the attribute as a call for the setter
# Then in the setter, you cannot use the return value with the exact name
# of the setter, so this is why you call the setter first_name and the attribute
# being set _first_name
class Person2(Person):
    def __init__(self, first_name):
        self.first_name = first_name

guido2 = Person2(2)

TypeError: Expected a string

In [5]:
ignacio = Person2('Ignacio')
print(ignacio.first_name, ignacio._first_name)

Ignacio Ignacio


In [64]:
guido.first_name = 'Guidoo'

In [65]:
guido.first_name

'Guidoo'

In [60]:
del guido.first_name

AttributeError: Can't delete attribute

In [17]:
# Invoke a method from the parent class: super()

# Metaclasses and interfaces: use abc library and ABDMeta, abstractmethod
# There are commonly used to force the inherited classes follow some rules
# I mean, they are used as templates for other classes.
# abd stands for Abstract Base Class

# Abstract methods MUST be implemented in the subclasses
# Metaclasses cant be instantiated 

from abc import ABCMeta, abstractmethod

class Pizza_interface(metaclass = ABCMeta):

    @abstractmethod
    def cook(self):
        pass

    # @property
    # @abstractmethod
    # def ingredients(self):
    #     return self._ingredients

    # @ingredients.setter
    # @abstractmethod
    # def ingredients(self, ingredients):
    #     if isinstance(ingredients, list):
    #         self._ingredients = ingredients
    #     else:
    #         raise(TypeError, f'Ingredients must be a list, not {type(ingredients)}')


class PizzaBBQ(Pizza_interface):

    INGREDIENTS = ['jam', 'meat', 'bacon', 'bbq_sauce']

    def __init__(self):
        self.ingredients = list()

    # def cook(self):
    #     self.ingredients = list(self.INGREDIENTS)

    def __repr__(self):
        return f"{type(self)}, ingredients: {self.ingredients}"


In [18]:
# The subclass must implement all abstract methods:
pizza_bbq_1 = PizzaBBQ()

TypeError: Can't instantiate abstract class PizzaBBQ with abstract methods cook

In [19]:
# Meta class cant be instantiated:
pizza = Pizza_interface()

TypeError: Can't instantiate abstract class Pizza_interface with abstract methods cook

In [24]:
class PizzaBBQ(Pizza_interface):

    INGREDIENTS = ['jam', 'meat', 'bacon', 'bbq_sauce']

    def __init__(self):
        self.ingredients = list()

    def cook(self):
        self.ingredients = list(self.INGREDIENTS)

    def __repr__(self):
        return f"{type(self)}, ingredients: {self.ingredients}"

In [25]:
pizza_bbq_2 = PizzaBBQ()
pizza_bbq_2

<class '__main__.PizzaBBQ'>, ingredients: []

In [26]:
pizza_bbq_2.cook()
pizza_bbq_2

<class '__main__.PizzaBBQ'>, ingredients: ['jam', 'meat', 'bacon', 'bbq_sauce']

In [15]:
# Descriptors: used when chekcs or assertions have to be placed on instance attributes

class Descriptor():
    def __init__(self, name = None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class TypedInt(Descriptor):
    expected_type = int
    def __set__(self, instance, value):
        super(TypedInt, self).__set__(instance, value)
        if not isinstance(value, self.expected_type):
            raise TypeError(f'expected {self.expected_type}, got {type(value)}')

In [1]:
class RandomClass():
    random_attribute = TypedInt()
    def __init__(self, random_attribute):
        self.random_attribute = random_attribute

    @property
    def random_attribute(self):
        return self._random_attribute

    @random_attribute.setter
    def random_attribute(self, random_attribute):
        self._random_attribute = random_attribute


In [14]:
type(None)

NoneType

In [4]:
coche_1.__dict__

{'brand': 'Ford'}

In [12]:
coche_1.__hash__(), id(coche_1)

(174007664388, 2784122630208)

In [5]:
dir(coche_1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'brand']